In [4]:
import cv2
import pyocr
import pyocr.builders
import pytesseract
import time
import re
import speech_recognition as sr
import pyttsx3
import threading
import torch
from PIL import Image

In [ ]:
# Configure Tesseract OCR (Update path if needed)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Initialize OCR tool
tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("No OCR tool found.")
    exit()
tool = tools[0]  # Use first available OCR tool

# Initialize text-to-speech engine
engine = pyttsx3.init()
recognizer = sr.Recognizer()
stop_flag = False  # Flag to control thread execution
triggered = False  # Flag to check if "read" command was spoken

# Open webcam first to show the reading area
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS, 30)  # Optimize for smoother video
if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

def text_to_speech(text):
    """Convert text to speech using pyttsx3."""
    print(f"Speaking: {text}")
    engine.say(text)
    engine.runAndWait()

def is_valid_text(text):
    """Ensures detected text is meaningful and not just single letters or artifacts."""
    text = text.strip()
    if len(text) < 2 or re.fullmatch(r"[^A-Za-z]+", text):  # Reject short words & symbols
        return False
    return True

def listen_for_trigger():
    """Listen for the trigger word 'read' to activate OCR and TTS."""
    global stop_flag, triggered
    while not stop_flag:
        with sr.Microphone() as source:
            recognizer.adjust_for_ambient_noise(source, duration=1.5)  # Increase duration
            print("Listening for trigger word 'read'...")
            try:
                audio = recognizer.listen(source)  # No timeout to avoid missing speech
                spoken_text = recognizer.recognize_google(audio).lower()
                if "read" in spoken_text:
                    print("Trigger word detected!")
                    triggered = True
            except sr.UnknownValueError:
                print("Could not understand audio.")
            except sr.RequestError:
                print("Speech recognition service unavailable.")

def process_frame(frame):
    """Process a video frame for OCR detection using PyOCR."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    pil_image = Image.fromarray(gray)
    
    # Use GPU acceleration if available
    if torch.cuda.is_available():
        device = torch.device("cuda")
        tensor_frame = torch.tensor(gray, device=device)
        gray = tensor_frame.cpu().numpy()
    
    # Use PyOCR for text detection
    detected_text = tool.image_to_string(
        pil_image, builder=pyocr.builders.TextBuilder()
    )
    
    # Validate detected words
    valid_words = [word for word in detected_text.split() if is_valid_text(word)]
    return " ".join(valid_words)

# Start trigger listening in a separate thread
print("Press 'q' to quit.")
trigger_thread = threading.Thread(target=listen_for_trigger)
trigger_thread.daemon = True  # Ensures thread closes with the program
trigger_thread.start()

Press 'q' to quit.


Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Could not understand audio.
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Trigger word detected!
Listening for trigger word 'read'...
Could not understand audio.
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Could not understand audio.
Listening for trigger word 'read'...
Could not understand audio.
Listening for trigger word 'read'...
Trigger word detected!
Listening for trigger word 'read'...
Trigger word detected!
Listening for trigger word 'read'...
Could not understand audio.
Listening for trigger word 'read'...
Trigger word detected!
Listening for trigger word 'read'...
Could not understand audio.
Listening for trigger word 'read'...
Could not understand audio.
Listening for trigger word 'read'...
Could not understand audio.
Listening for trigger word 'read'...
Could not understa

In [6]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture video. Exiting...")
        break
    
    # If trigger word is detected, process the frame
    if triggered:
        print("Processing text...")
        full_text = process_frame(frame)
        if full_text:
            print(f"Detected Text: {full_text}")
            text_to_speech(full_text)
            time.sleep(2)  # Prevent repeating too fast
        triggered = False  # Reset trigger after processing
    
    # Display bounding boxes for detected words
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    word_boxes = tool.image_to_string(pil_image, builder=pyocr.builders.WordBoxBuilder())
    
    for word_box in word_boxes:
        top_left, bottom_right = word_box.position
        top_left = tuple(map(int, top_left))
        bottom_right = tuple(map(int, bottom_right))
        
        cv2.rectangle(frame, top_left, bottom_right, (0, 0, 255), 2)
        cv2.putText(frame, word_box.content, (top_left[0], max(0, top_left[1] - 10)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    # Display video feed
    cv2.imshow("Live OCR Feed - Say 'Read' to Capture", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        stop_flag = True
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
trigger_thread.join()

Processing text...
Detected Text: Microsoft's Soul and Imagine Better Future for Everyone
Speaking: Microsoft's Soul and Imagine Better Future for Everyone
Processing text...
Processing text...
Processing text...
Detected Text: hope you were only UYINg tO protec “T'm okay," tel “Tl be <nnouncing: the team tomorrow, D0inted. three feet” “T understand,™ tter D, hopin I've Ruessed Set picked relief, Was over, Yes. Four team members and picase,” Mr Dimminaaae us, “Make am tired, and Dive Our Paper, then Cover it with the pass it Up to me €Ts and scribble their names n€ print button on my Medifulkenal nswers emerges from the side. Mp ick to where sit and rips it off Hage here,” he tells the class. “Your parem time to pick you up, but ifanyene ride, let me know. won't lene ryone has safely left school gram tone out. know my mon will
Speaking: hope you were only UYINg tO protec “T'm okay," tel “Tl be <nnouncing: the team tomorrow, D0inted. three feet” “T understand,™ tter D, hopin I've Ruesse